# Dose–Response Relationships  

You will need to edit the file Quiz.md to get autograded for this assignment.  

The first part of the assignment will have you performing analysis on Vanderbilt's Thunor instance.  

You can access the plots for afatinib treatment using DIP rate or viability at 72 h by following [this link](https://thunor.app.vanderbilt.edu/plots?dataset=13&colsLg=3&colsMd=2&plotdata=plotType%3Ddrc%26datasetId%3D13%26dataset2Id%3D%26useCellLineTags%3Doff%26c%3D8%26c%3D9%26c%3D10%26c%3D11%26c%3D12%26c%3D17%26c%3D18%26useDrugTags%3Doff%26d%3D5%26colorBy%3Doff%26drMetric%3Ddip%26drcType%3Drel&plotdata=plotType%3Ddrc%26datasetId%3D13%26dataset2Id%3D%26useCellLineTags%3Doff%26c%3D8%26c%3D9%26c%3D10%26c%3D11%26c%3D12%26c%3D17%26c%3D18%26useDrugTags%3Doff%26d%3D5%26colorBy%3Doff%26drMetric%3Dviability%26drcType%3Drel)  

Then you can add new plots as needed to answer the questions in the Quiz.md. You must modify the Quiz.md file to get your grade (either modify the file directly on GitHub or upload to your assignment repository.)  


You can learn more about Thunor web and how to use it in [this tutorial](https://youtu.be/q4LQCjuBnmg).  


## Assignment

Task 1) 
Got to [thunor.app.vanderbilt.edu](https://thunor.app.vanderbilt.edu/plots?dataset=13&colsLg=3&colsMd=2&plotdata=plotType%3Ddrc%26datasetId%3D13%26dataset2Id%3D%26useCellLineTags%3Doff%26c%3D8%26c%3D9%26c%3D10%26c%3D11%26c%3D12%26c%3D17%26c%3D18%26useDrugTags%3Doff%26d%3D5%26colorBy%3Doff%26drMetric%3Ddip%26drcType%3Drel&plotdata=plotType%3Ddrc%26datasetId%3D13%26dataset2Id%3D%26useCellLineTags%3Doff%26c%3D8%26c%3D9%26c%3D10%26c%3D11%26c%3D12%26c%3D17%26c%3D18%26useDrugTags%3Doff%26d%3D5%26colorBy%3Doff%26drMetric%3Dviability%26drcType%3Drel) and compare the responses of all cell lines to afatinib using DIP rate or 72 h viability as the effect metric.

Task 2) Complete all tasks within this notebook  

Task 3) Edit and submit the Quiz.md file in your assigment repository (you will need to refer to thunor@VU and this notebook).  

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    try:
        import thunor
    except:
        !pip install thunor

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
import thunor
from thunor.io import read_hdf
from thunor.dip import dip_rates
from thunor.curve_fit import fit_params
%matplotlib inline

#### Define model functions

In [ ]:
def ll4(x,h,E0,Emax,EC50):
    '''
    Reformulation of the LL4 function
     - h: Hill coefficient
     - E0: no-drug effect
     - Emax: max effect
     - EC50: Concentration at midpoint between E0 and Emax
     '''
    return(Emax + (E0-Emax) / (1+np.exp(h*(np.log(x/EC50)))) )

def ll3(x,b,d,e):
    '''
    3-parameter log-logistic function with no/minimal effect set to a value of 1
     - b: Hill coefficient
     - d: Emax
     - e: EC50
     '''
    return(d + (1-d) / ( 1+np.exp( b*np.log(x/e)) ) )

#### Load all PC9 data using Thunor

In [ ]:
if IN_COLAB and not os.path.exists("./data/HTS001.h5"):
    if not os.path.exists("./data"):
        !mkdir data
    !wget -O ./data/HTS001.h5 https://github.com/VU-CSP/quantbio-assignments/raw/main/data/HTS001.h5
    


#### Description of HTS001.h5 dataset  

* PC9 lung adenocarcinoma cell line and six PC9 variants, each treated with 14 different drugs
* All cell lines were genetically modified to express a nuclear-localized fluorescent protein (H2BmRFP) and assessed by live-cell fluorescence microscopy in multiwell plates
* No cell death indicator was used in these experiments (i.e., some of the cell counts may be of dead cells that retained fluorescent nuclear signal)

In [ ]:
hts001 = read_hdf("./data/HTS001.h5")
hts001

### Calculate DIP rates and fit parameters for all conditions using Thunor

In [ ]:
ctrl_diprates, expt_diprates = dip_rates(hts001)
fit_p = fit_params(ctrl_diprates, expt_diprates)

### Examine the fit parameters using `head()`

In [ ]:
fit_p.head()

### Download DIP rate data (automatically fit to time course data by Thunor)

In [ ]:
if not os.path.exists("./data/hts001_diprates.tsv"):
    !wget -O "./data/hts001_diprates.tsv" https://thunor.app.vanderbilt.edu/dataset/13/download/dip_rates

hts001_diprates = pd.read_csv("./data/hts001_diprates.tsv", sep='\t')
hts001_diprates.rename(columns={'cell.line': 'cell_line', 'drug1':'drug', 'drug1.conc':'conc'}, inplace=True)
hts001_diprates.head()


### Extract only cell count data from afatinib treatment
Start from full dataset (`hts001`)

In [ ]:
afat = hts001.filter(drugs=['afatinib'])
afat

### Load thunor plotting functions

In [ ]:
from thunor.plots import plot_drc, plot_drc_params, plot_time_course, plot_ctrl_dip_by_plate, plot_plate_map


### Filter fit parameters for all cell lines treated with afatinib, and plot dose–response curves

In [ ]:
fit_p_afat = fit_p[fit_p.index.isin(['afatinib'], level='drug')]
fig = plot_drc(fit_p_afat)
fig.update_layout(
    autosize=False,
    width=400,
    height=400,)

### Filter fit parameters by cell line (`BR1`) and drug (`paclitaxel`)
and display result

In [ ]:
fp_br1_pacl = fit_p[fit_p.index.isin(['BR1'], level='cell_line') & \
               fit_p.index.isin(['paclitaxel'], level='drug')]
fp_br1_pacl

### Show the result of Thunor-fit DIP rates
Saved in `expt_diprates`

In [ ]:
expt_diprates

### Filter DIP rates by cell line (`BR1`) and drug (`paclitaxel`)

In [ ]:
expt_diprates[expt_diprates.index.isin(['BR1'], level='cell_line')]

In [ ]:
br1_afat = afat.filter(cell_lines=['BR1'])

### Plot the time course data of afatinib on BR1 cells
Use `log_yaxis=True` to plot population doublings.

In [ ]:
fig = plot_time_course(br1_afat, log_yaxis=True)
fig.update_layout(
    autosize=False,
    width=400,
    height=400,)

### Writing and reading cell count data using Vanderbilt's HTS format
To faciltate executing Python code, we will replace the `.` in column names with `_`.

In [ ]:
from pandas.io.parsers.readers import read_csv
if not os.path.exists("./data/afatinib.csv"):
    thunor.io.write_vanderbilt_hts(afat, filename="./data/afatinib.csv")
a = read_csv("./data/afatinib.csv")
a.rename(columns={'cell.line': 'cell_line', 'cell.count': 'cell_count', 'drug1.conc': 'drug1_conc', 'drug1.units': 'drug1_units'}, inplace=True)
a.head()


### Assemble basic DataFrame of DIP rates of control and paclitaxel-treated BR1 cells


In [ ]:
diprates_br1_pacl = hts001_diprates[np.isin(hts001_diprates['drug'], ['paclitaxel']) ]
diprates_br1_pacl = diprates_br1_pacl[diprates_br1_pacl['cell_line'] == 'BR1']
# add control diprates
br1_diprates_ctrl = hts001_diprates[np.logical_and(hts001_diprates['conc'].isna(), hts001_diprates['cell_line']=='BR1')]
diprates_br1_pacl = diprates_br1_pacl.append(br1_diprates_ctrl)


### Add log10([drug]) values to facilitate manual plotting

In [ ]:
l10_conc = np.log10(diprates_br1_pacl['conc'][diprates_br1_pacl['conc'].notna()])
min_conc = min(l10_conc)-1
n_ctrl_wells = len( diprates_br1_pacl['conc'][diprates_br1_pacl['conc'].isna()])
l10_conc = np.append(l10_conc,np.repeat(min_conc,n_ctrl_wells))
diprates_br1_pacl['l10_conc'] = l10_conc
diprates_br1_pacl.head()

### Calculate response ratio of DIP rates
Divide all DIP rate values by the mean of the control values

In [ ]:
diprates_br1_pacl["resp_ratio"] = diprates_br1_pacl["dip_rate"] / diprates_br1_pacl[diprates_br1_pacl["conc"].isnull()]["dip_rate"].mean()

### Show the structure of the DataFrame

In [ ]:
diprates_br1_pacl

### Plot the DIP rate values of BR1 cells treated with paclitaxel using Seaborn

In [ ]:
sns.lmplot(x='l10_conc', y='dip_rate', data=diprates_br1_pacl, fit_reg=False)

#### Perform nonlinear regression on data using the `ll4` model  
Let's first look at each of the dates independently to assess consistency.  
#### Use `scipy.optimize.curve_fit` to perform nonlinear regression (fit model parameters)  


In [ ]:
fitCoefs, covMatrix = opt.curve_fit(ll4, 
                                    10**(diprates_br1_pacl['l10_conc']), 
                                    diprates_br1_pacl['dip_rate'],
                                    p0=[1,
                                        np.max(diprates_br1_pacl['dip_rate']),
                                        np.min(diprates_br1_pacl['dip_rate']), 
                                        1e-9])
fitCoefs

# TASK

What is the value that represents the $EC_{50}$ of paclitaxel's effects on BR1 cells? Copy it and save as the answer to Quiz question #6.

### NOTE: Values from paclitaxel of ~6.2e-8 M were found to be artifactual
Drug concentration at this value is inaccurate. Data can be removed to allow more accurate fitting of dose-response model

In [ ]:
# remove artifact points
diprates_br1_pacl_drop62 = diprates_br1_pacl[np.logical_or(diprates_br1_pacl['conc'] < 2e-8, diprates_br1_pacl['conc'] > 7e-8)]

# TASK
### Fit the cleaned data with the `ll4` model
NOTE: to get the model to fit you may need some starting parameters (`p0`)

In [ ]:
fitCoefs_dropped, covMatrix_dropped = 0,0  # Put your code here (replace the 0,0 values)

fitCoefs_dropped  # show the values of the fit parameters; leave this line in

# TASK

Copy the value that represents the corrected $EC_{50}$ of paclitaxel's effects on BR1 cells and save as the answer to Quiz question #7.

#### Plot the data and predicted model fits using Seaborn
NOTE: You will see two curves if you properly fit the cleaned data with the `ll4` model and saved the fit parameters in `fitCoefs_dropped`  

In [ ]:
myconc = np.linspace(-12,-5,100)
sns.lmplot(x='l10_conc', y='dip_rate', data=diprates_br1_pacl, fit_reg=False)
if type(fitCoefs_dropped)!=int:
    sns.lineplot(x=myconc, y=[ll4(x, *fitCoefs_dropped) for x in 10**myconc], legend=False)
sns.lineplot(x=myconc, y=[ll4(x, *fitCoefs) for x in 10**myconc], legend=False, hue=2)

#### Perform nonlinear regression on data using `ll3` model  
Use `resp_ratio` values. 


In [ ]:
fitCoefs_ratio, covMatrix_ratio = opt.curve_fit(ll3, 10**(diprates_br1_pacl_drop62['l10_conc']), 
                                    diprates_br1_pacl_drop62['resp_ratio'])
fitCoefs_ratio

# Task
What is the $E_{max}$ value from the `ll3` model parameters fit to response ratios of BR1 + paclitaxel? Copy it and save the answer to Quiz question #8. 

#### Plot the data and predicted model fits using Seaborn

In [ ]:
a = sns.lmplot(x='l10_conc', y='resp_ratio', data=diprates_br1_pacl_drop62, fit_reg=False)
a.set(ylim=(-1, 1.25))
l = sns.lineplot(x=[i for i in myconc], y=[ll3(10**i,*fitCoefs_ratio) for i in myconc], legend=False)

In [ ]:
print(f"The EC50 of paclitaxel on BR1 cells is {np.round(fitCoefs_ratio[2]*1e9,3)} nM")